# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'09-05-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'09-05-2020'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-09-06 04:28:33,33.93911,67.709953,38324,1409,30082,6833.0,Afghanistan,98.447555,3.676547
1,NaN,NaN,NaN,Albania,2020-09-06 04:28:33,41.15330,20.168300,10102,312,5976,3814.0,Albania,351.032038,3.088497
2,NaN,NaN,NaN,Algeria,2020-09-06 04:28:33,28.03390,1.659600,46071,1549,32481,12041.0,Algeria,105.062495,3.362202
3,NaN,NaN,NaN,Andorra,2020-09-06 04:28:33,42.50630,1.521800,1215,53,928,234.0,Andorra,1572.510192,4.362140
4,NaN,NaN,NaN,Angola,2020-09-06 04:28:33,-11.20270,17.873900,2935,117,1192,1626.0,Angola,8.930129,3.986371


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/27/20,8/28/20,8/29/20,8/30/20,8/31/20,9/1/20,9/2/20,9/3/20,9/4/20,9/5/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,38129,38140,38143,38162,38165,38196,38243,38288,38304,38324
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,9083,9195,9279,9380,9513,9606,9728,9844,9967,10102
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,43016,43403,43781,44146,44494,44833,45158,45469,45773,46071


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/27/20,8/28/20,8/29/20,8/30/20,8/31/20,9/1/20,9/2/20,9/3/20,9/4/20,9/5/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1401,1402,1402,1402,1402,1406,1409,1409,1409,1409
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,266,271,275,280,284,290,296,301,306,312
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,1475,1483,1491,1501,1510,1518,1523,1529,1539,1549


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/27/20,8/28/20,8/29/20,8/30/20,8/31/20,9/1/20,9/2/20,9/3/20,9/4/20,9/5/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,29046,29059,29063,29089,29089,29231,29315,29390,29713,30082
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,4791,4923,5020,5139,5214,5441,5582,5732,5882,5976
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,30157,30436,30717,30978,31244,31493,31746,32006,32259,32481


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/27/20,8/28/20,8/29/20,8/30/20,8/31/20,9/1/20,9/2/20,9/3/20,9/4/20,9/5/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1284,1296,1309,1345,1348,1354,1345,1349,1355,1371
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,4199,4230,4307,4402,4422,4445,4458,4495,4513,4542
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,614,620,624,628,628,629,617,614,616,616


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/27/20,8/28/20,8/29/20,8/30/20,8/31/20,9/1/20,9/2/20,9/3/20,9/4/20,9/5/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,22,22,22,22,22,23,23,23,23,23
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,34,35,36,36,38,38,38,40,42,42
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,7,7,7,7,7,7,7,7,7,7


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
631,1001.0,Autauga,Alabama,US,2020-09-06 04:28:33,32.539527,-86.644082,1371,23,0,1348.0,"Autauga, Alabama, US",2453.954787,1.677608
668,1075.0,Lamar,Alabama,US,2020-09-06 04:28:33,33.779950,-88.096680,280,2,0,278.0,"Lamar, Alabama, US",2028.250634,0.714286
669,1077.0,Lauderdale,Alabama,US,2020-09-06 04:28:33,34.901719,-87.656247,1467,25,0,1442.0,"Lauderdale, Alabama, US",1582.029354,1.704158


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,6244970,188538,2302187
Brazil,4123000,126203,3498999
India,4113811,70626,3180865


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,6244970,188538,2302187,3754245,2020-09-06 04:28:33,37.923092,-91.345036
Brazil,4123000,126203,3498999,497798,2020-09-06 04:28:33,-12.669522,-48.480493
India,4113811,70626,3180865,862320,2020-09-06 04:28:33,23.088275,81.806127
Russia,1017131,17707,835884,163540,2020-09-06 04:28:33,54.546312,62.120860
Peru,676848,29554,498523,148771,2020-09-06 04:28:33,-10.637324,-75.277744


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,735314,13709,0
Texas,655815,13576,0
Florida,643867,11811,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,735314,13709,0,721605,2020-09-06 04:28:33,37.843962,-120.728594
Texas,655815,13576,0,642239,2020-09-06 04:28:33,31.660643,-98.653069
Florida,643867,11811,0,632056,2020-09-06 04:28:33,28.940755,-82.700744
New York,438772,32987,0,405785,2020-09-06 04:28:33,42.544151,-75.474183
Georgia,281548,5977,0,275571,2020-09-06 04:28:33,32.808532,-83.577430


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,247542,6000,0
Florida,Miami-Dade,161018,2650,0
Arizona,Maricopa,135856,3088,0
Illinois,Cook,130616,5092,0
Texas,Harris,110762,2327,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,247542,6000,0,241542,2020-09-06 04:28:33,34.308284,-118.228241,6037.0
Florida,Miami-Dade,161018,2650,0,158368,2020-09-06 04:28:33,25.611236,-80.551706,12086.0
Arizona,Maricopa,135856,3088,0,132768,2020-09-06 04:28:33,33.348359,-112.491815,4013.0
Illinois,Cook,130616,5092,0,125524,2020-09-06 04:28:33,41.841448,-87.816588,17031.0
Texas,Harris,110762,2327,0,108435,2020-09-06 04:28:33,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-03,38288,9844,45469,1199,2805,95,451198,44271,26136,28372,...,342708,1636,42688,49877,1046,24471,10,1979,12523,6678
2020-09-04,38304,9967,45773,1215,2876,95,461882,44461,26207,28729,...,344686,1653,42998,50973,1049,25142,10,1983,12639,6837
2020-09-05,38324,10102,46071,1215,2935,95,471806,44649,26278,29087,...,346513,1669,43293,52165,1049,25575,10,1983,12709,6837


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-03,1409,301,1529,53,113,3,9361,887,737,735,...,41616,44,331,402,35,167,1,571,292,206
2020-09-04,1409,306,1539,53,115,3,9623,891,748,735,...,41626,45,338,412,35,170,1,572,292,206
2020-09-05,1409,312,1549,53,117,3,9739,895,753,735,...,41638,45,343,420,35,177,1,572,292,206


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-03,29390,5732,32006,909,1144,91,322461,38855,21912,24171,...,1750,1437,40081,40574,755,16095,8,1180,11562,5263
2020-09-04,29713,5882,32259,928,1167,91,331621,39257,22170,24513,...,1756,1446,40392,41249,772,16437,8,1194,11572,5345
2020-09-05,30082,5976,32481,928,1192,91,340381,39823,22328,24828,...,1806,1459,40774,42006,805,16661,8,1197,11668,5345


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/27/20,8/28/20,8/29/20,8/30/20,8/31/20,9/1/20,9/2/20,9/3/20,9/4/20,9/5/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1284,1296,1309,1345,1348,1354,1345,1349,1355,1371
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,266,266,268,271,273,277,277,276,278,280
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,1373,1384,1398,1405,1411,1424,1433,1443,1454,1467
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,432,433,442,459,463,466,469,486,488,501
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,3031,3091,3200,3247,3348,3395,3371,3384,3417,3466


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-03,129285,5464,0,203953,63081,726018,58267,53209,17653,49,...,159546,645422,53326,1637,1150,123637,75856,10855,77856,3941
2020-09-04,130393,5584,0,204681,64175,730662,58267,53365,17752,49,...,160597,651350,53839,1642,1150,124738,76335,11042,79354,3990
2020-09-05,131803,5677,0,205517,64690,735314,58966,53365,17892,49,...,162362,655815,54272,1647,1167,125681,76836,11293,80300,3990


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-09-03        1349    4495     614  542   1037     541    841    2447   
2020-09-04        1355    4513     616  545   1049     541    848    2503   
2020-09-05        1371    4542     616  550   1062     544    853    2547   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-09-03          850      354  ...     171      7      157       49   
2020-09-04          850      362  ...     172      7      170       48   
2020-09-05          853      364  ...     172      7      170       48   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-09-03            304   435   305          0      109     20  
2020-09-04            307   440   308          0      110     20  
2020-09-05            307   440   308          0      110     20  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-03,2233,40,0,5130,861,13493,1955,4468,606,0,...,1815,13296,414,58,16,2652,1945,240,1146,41
2020-09-04,2266,40,0,5171,873,13638,1955,4468,606,0,...,1837,13426,419,58,16,2662,1953,245,1153,42
2020-09-05,2275,42,0,5207,882,13709,1971,4468,608,0,...,1862,13576,420,58,16,2677,1953,246,1168,42


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-09-03          23      40       7    6     11      13     36      31   
2020-09-04          23      42       7    6     12      13     37      31   
2020-09-05          23      42       7    6     12      13     37      31   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-09-03           39       11  ...       1      1        1        1   
2020-09-04           40       11  ...       1      1        1        1   
2020-09-05           40       11  ...       1      1        1        1   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-09-03              2     1     2          0        6      0  
2020-09-04              2     1     2          0        6      0  
2020-09-05              2     1     2          0        6      0  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-03,0.001177,0.011924,0.006887,0.000000,0.010083,0.010638,0.027383,0.004447,0.003340,0.014409,...,0.005218,0.006150,0.005915,0.020334,0.000000,0.024963,0.0,0.001518,0.008699,0.006026
2020-09-04,0.000418,0.012495,0.006686,0.013344,0.025312,0.000000,0.023679,0.004292,0.002717,0.012583,...,0.005772,0.010391,0.007262,0.021974,0.002868,0.027420,0.0,0.002021,0.009263,0.023810
2020-09-05,0.000522,0.013545,0.006510,0.000000,0.020515,0.000000,0.021486,0.004228,0.002709,0.012461,...,0.005300,0.009679,0.006861,0.023385,0.000000,0.017222,0.0,0.000000,0.005538,0.000000


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-03,0.0,0.016892,0.003940,0.0,0.008929,0.0,0.026651,0.003394,0.087021,0.001362,...,0.000337,0.000000,0.012232,0.010050,0.029412,0.030864,0.0,0.000000,0.0,0.0
2020-09-04,0.0,0.016611,0.006540,0.0,0.017699,0.0,0.027988,0.004510,0.014925,0.000000,...,0.000240,0.022727,0.021148,0.024876,0.000000,0.017964,0.0,0.001751,0.0,0.0
2020-09-05,0.0,0.019608,0.006498,0.0,0.017391,0.0,0.012054,0.004489,0.006684,0.000000,...,0.000288,0.000000,0.014793,0.019417,0.000000,0.041176,0.0,0.000000,0.0,0.0


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-03,0.002558,0.026872,0.008190,0.000000,0.026009,0.0,0.021966,0.005798,0.010235,0.014736,...,0.004592,0.002791,0.010513,0.016586,0.012064,0.039527,0.0,0.004255,0.005916,0.002476
2020-09-04,0.010990,0.026169,0.007905,0.020902,0.020105,0.0,0.028407,0.010346,0.011774,0.014149,...,0.003429,0.006263,0.007759,0.016636,0.022517,0.021249,0.0,0.011864,0.000865,0.015580
2020-09-05,0.012419,0.015981,0.006882,0.000000,0.021422,0.0,0.026416,0.014418,0.007127,0.012850,...,0.028474,0.008990,0.009457,0.018352,0.042746,0.013628,0.0,0.002513,0.008296,0.000000


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-03,0.008157,0.019023,NaN,0.005383,0.015601,0.006567,0.004621,0.001902,0.005926,0.0,...,0.010866,0.005557,0.009541,0.000000,0.005245,0.009133,0.006355,0.018484,0.009426,0.007671
2020-09-04,0.008570,0.021962,NaN,0.003569,0.017343,0.006397,0.000000,0.002932,0.005608,0.0,...,0.006587,0.009185,0.009620,0.003054,0.000000,0.008905,0.006315,0.017227,0.019241,0.012433
2020-09-05,0.010813,0.016655,NaN,0.004084,0.008025,0.006367,0.011996,0.000000,0.007886,0.0,...,0.010990,0.006855,0.008042,0.003045,0.014783,0.007560,0.006563,0.022731,0.011921,0.000000


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-09-03      0.002974  0.008300 -0.004862  0.001848  0.000965  0.001852   
2020-09-04      0.004448  0.004004  0.003257  0.005535  0.011572  0.000000   
2020-09-05      0.011808  0.006426  0.000000  0.009174  0.012393  0.005545   

Province_State                                          ...   Wyoming         \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park Platte   
2020-09-03      0.002384  0.014090 -0.012776  0.000000  ...  0.005882    0.0   
2020-09-04      0.008323  0.022885  0.000000  0.022599  ...  0.005848    0.0   
2020-09-05      0.005896  0.017579  0.003529  0.005525  ...  0.000000    0.0   

Province_State                                                                \
Admin2          Sheridan  Sublette Sweetwater     Teton     Uinta Unassigned   
2020-09-03      0.019481  0.020833   0.000000  0.004619  0.006601        NaN   
2020-09-04      0.082803 -0.020408   0.009868  0.011494  0.009836        NaN   
2020-09-05      0.000000  0.000000   0.000000  0.000000  0.000000        NaN   

Province_State                      
Admin2          Washakie    Weston  
2020-09-03      0.000000  0.052632  
2020-09-04      0.009174  0.000000  
2020-09-05      0.000000  0.000000  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-03,0.007217,0.025641,NaN,0.012833,0.023781,0.013216,0.001537,0.000224,0.0000,NaN,...,0.010017,0.014652,0.009756,0.0,0.066667,0.004165,0.005168,0.030043,0.003503,0.00000
2020-09-04,0.014778,0.000000,NaN,0.007992,0.013937,0.010746,0.000000,0.000000,0.0000,NaN,...,0.012121,0.009777,0.012077,0.0,0.000000,0.003771,0.004113,0.020833,0.006108,0.02439
2020-09-05,0.003972,0.050000,NaN,0.006962,0.010309,0.005206,0.008184,0.000000,0.0033,NaN,...,0.013609,0.011172,0.002387,0.0,0.000000,0.005635,0.000000,0.004082,0.013010,0.00000


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                \
Admin2         Autauga   Baldwin Barbour      Bibb    Blount Bullock   
2020-09-03         0.0  0.052632     0.0 -0.142857  0.000000     0.0   
2020-09-04         0.0  0.050000     0.0  0.000000  0.090909     0.0   
2020-09-05         0.0  0.000000     0.0  0.000000  0.000000     0.0   

Province_State                                        ... Wyoming         \
Admin2            Butler  Calhoun  Chambers Cherokee  ...    Park Platte   
2020-09-03      0.000000 -0.03125  0.000000      0.0  ...     0.0    0.0   
2020-09-04      0.027778  0.00000  0.025641      0.0  ...     0.0    0.0   
2020-09-05      0.000000  0.00000  0.000000      0.0  ...     0.0    0.0   

Province_State                                                               \
Admin2         Sheridan Sublette Sweetwater Teton Uinta Unassigned Washakie   
2020-09-03          0.0      0.0        0.0   0.0   0.0        NaN      0.0   
2020-09-04          0.0      0.0        0.0   0.0   0.0        NaN      0.0   
2020-09-05          0.0      0.0        0.0   0.0   0.0        NaN      0.0   

Province_State         
Admin2         Weston  
2020-09-03        NaN  
2020-09-04        NaN  
2020-09-05        NaN  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-03,0.001027,0.011967,0.007240,0.007173,0.015496,0.005319,0.026021,0.003881,0.003804,0.012318,...,0.004757,0.007551,0.006742,0.021352,0.000793,0.024861,5.882173e-24,0.003154,0.009070,0.008318
2020-09-04,0.000723,0.012231,0.006963,0.010259,0.020404,0.002660,0.024850,0.004086,0.003260,0.012451,...,0.005264,0.008971,0.007002,0.021663,0.001831,0.026141,2.941087e-24,0.002588,0.009166,0.016064
2020-09-05,0.000622,0.012888,0.006737,0.005129,0.020459,0.001330,0.023168,0.004157,0.002985,0.012456,...,0.005282,0.009325,0.006931,0.022524,0.000915,0.021681,1.470543e-24,0.001294,0.007352,0.008032


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-03,0.000908,0.018208,0.004219,2.292486e-09,0.013450,1.807675e-43,0.025141,0.003282,0.053623,0.000853,...,0.000251,0.000389,0.012937,0.013161,0.019885,0.027609,0.0,0.001647,0.003233,0.007001
2020-09-04,0.000454,0.017410,0.005380,1.146243e-09,0.015575,9.038375e-44,0.026565,0.003896,0.034274,0.000426,...,0.000246,0.011558,0.017042,0.019018,0.009943,0.022786,0.0,0.001699,0.001617,0.003501
2020-09-05,0.000227,0.018509,0.005939,5.731216e-10,0.016483,4.519188e-44,0.019310,0.004193,0.020479,0.000213,...,0.000267,0.005779,0.015918,0.019218,0.004971,0.031981,0.0,0.000850,0.000808,0.001750


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-03,0.002692,0.027759,0.008189,0.000807,0.022792,0.000703,0.022756,0.007224,0.008589,0.012953,...,0.011759,0.006801,0.008659,0.020472,0.017479,0.026680,3.446586e-26,0.009463,0.004933,0.004298
2020-09-04,0.006841,0.026964,0.008047,0.010854,0.021448,0.000351,0.025581,0.008785,0.010182,0.013551,...,0.007594,0.006532,0.008209,0.018554,0.019998,0.023964,1.723293e-26,0.010664,0.002899,0.009939
2020-09-05,0.009630,0.021473,0.007464,0.005427,0.021435,0.000176,0.025998,0.011601,0.008654,0.013201,...,0.018034,0.007761,0.008833,0.018453,0.031372,0.018796,8.616465e-27,0.006588,0.005597,0.004970


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-03,0.007993,0.014662,NaN,0.003849,0.011889,0.006794,0.004628,0.002233,0.004519,1.137347e-52,...,0.010200,0.006149,0.008318,0.001885,0.005391,0.008508,0.005839,0.017679,0.008907,0.007837
2020-09-04,0.008282,0.018312,NaN,0.003709,0.014616,0.006595,0.002314,0.002582,0.005064,5.686733e-53,...,0.008394,0.007667,0.008969,0.002470,0.002696,0.008707,0.006077,0.017453,0.014074,0.010135
2020-09-05,0.009548,0.017483,NaN,0.003897,0.011320,0.006481,0.007155,0.001291,0.006475,2.843367e-53,...,0.009692,0.007261,0.008506,0.002757,0.008739,0.008133,0.006320,0.020092,0.012997,0.005068


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-09-03      0.001622  0.006919 -0.006676  0.004093  0.001305  0.001766   
2020-09-04      0.003035  0.005462 -0.001709  0.004814  0.006438  0.000883   
2020-09-05      0.007421  0.005944 -0.000855  0.006994  0.009416  0.003214   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-09-03      0.002085  0.010865 -0.009485  0.004065  ...  0.011410   
2020-09-04      0.005204  0.016875 -0.004743  0.013332  ...  0.008629   
2020-09-05      0.005550  0.017227 -0.000607  0.009428  ...  0.004314   

Province_State                                                         \
Admin2                Platte  Sheridan  Sublette Sweetwater     Teton   
2020-09-03      1.319249e-06  0.019511  0.010438   0.002436  0.004698   
2020-09-04      6.596247e-07  0.051157 -0.004985   0.006152  0.008096   
2020-09-05      3.298124e-07  0.025579 -0.002493   0.003076  0.004048   

Province_State                                           
Admin2             Uinta Unassigned  Washakie    Weston  
2020-09-03      0.007299  -0.495957  0.002319  0.208125  
2020-09-04      0.008568  -0.495957  0.005747  0.104059  
2020-09-05      0.004284  -0.495957  0.002873  0.052029  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-03,0.007824,0.019768,NaN,0.008206,0.025014,0.011802,0.001778,0.000198,0.000522,NaN,...,0.010299,0.013264,0.006246,3.293784e-11,0.042704,0.006918,0.004569,0.031901,0.005543,0.027034
2020-09-04,0.011301,0.009884,NaN,0.008099,0.019476,0.011274,0.000889,0.000099,0.000261,NaN,...,0.011210,0.011520,0.009162,1.646892e-11,0.021352,0.005344,0.004341,0.026367,0.005826,0.025712
2020-09-05,0.007636,0.029942,NaN,0.007531,0.014892,0.008240,0.004537,0.000050,0.001781,NaN,...,0.012410,0.011346,0.005774,8.234460e-12,0.010676,0.005490,0.002171,0.015224,0.009418,0.012856


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                              \
Admin2           Autauga   Baldwin       Barbour      Bibb    Blount   
2020-09-03      0.005682  0.030647  2.555056e-06 -0.058206  0.012292   
2020-09-04      0.002841  0.040323  1.277528e-06 -0.029103  0.051601   
2020-09-05      0.001420  0.020162  6.387640e-07 -0.014552  0.025800   

Province_State                                                            ...  \
Admin2               Bullock        Butler   Calhoun  Chambers  Cherokee  ...   
2020-09-03      6.935813e-07  2.129316e-10 -0.003397  0.000206  0.000872  ...   
2020-09-04      3.467907e-07  1.388889e-02 -0.001699  0.012923  0.000436  ...   
2020-09-05      1.733953e-07  6.944444e-03 -0.000849  0.006462  0.000218  ...   

Province_State Wyoming                                                  \
Admin2            Park Platte Sheridan Sublette Sweetwater Teton Uinta   
2020-09-03         0.0    0.0      0.0      0.0        0.0   0.0   0.0   
2020-09-04         0.0    0.0      0.0      0.0        0.0   0.0   0.0   
2020-09-05         0.0    0.0      0.0      0.0        0.0   0.0   0.0   

Province_State                             
Admin2         Unassigned Washakie Weston  
2020-09-03      -0.499985      0.0    NaN  
2020-09-04      -0.499985      0.0    NaN  
2020-09-05      -0.499985      0.0    NaN  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200905,AK,6567,378435,NaN,36.0,NaN,NaN,NaN,8.0,...,385002,2,0,433e94e907f6c4815829cdad34dbf4c296918fc6,0,0,0,0,0,NaN
1,20200905,AL,131803,846170,NaN,818.0,14914.0,NaN,1537.0,NaN,...,977973,9,113,b22e91650c4091000389f5e6f7d8cd4e4bb981eb,0,0,0,0,0,NaN
2,20200905,AR,64690,700721,NaN,385.0,4456.0,NaN,NaN,78.0,...,765411,9,34,943b6e2742aae6e78aa190fb590e34d8804ccbd1,0,0,0,0,0,NaN
3,20200905,AS,0,1514,NaN,NaN,NaN,NaN,NaN,NaN,...,1514,0,0,78615189f736f50add2170e1fb8468820dba264a,0,0,0,0,0,NaN
4,20200905,AZ,205516,1047259,NaN,702.0,21520.0,225.0,NaN,131.0,...,1252775,36,22,f2b8f923b223050916eb45768429c30f9dce719c,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200905,AK,6567.0,378435.0,NaN,36.0,NaN,NaN,NaN,8.0,...,385002,2,0,433e94e907f6c4815829cdad34dbf4c296918fc6,0,0,0,0,0,NaN
1,20200905,AL,131803.0,846170.0,NaN,818.0,14914.0,NaN,1537.0,NaN,...,977973,9,113,b22e91650c4091000389f5e6f7d8cd4e4bb981eb,0,0,0,0,0,NaN
2,20200905,AR,64690.0,700721.0,NaN,385.0,4456.0,NaN,NaN,78.0,...,765411,9,34,943b6e2742aae6e78aa190fb590e34d8804ccbd1,0,0,0,0,0,NaN
3,20200905,AS,0.0,1514.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1514,0,0,78615189f736f50add2170e1fb8468820dba264a,0,0,0,0,0,NaN
4,20200905,AZ,205516.0,1047259.0,NaN,702.0,21520.0,225.0,NaN,131.0,...,1252775,36,22,f2b8f923b223050916eb45768429c30f9dce719c,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-09-05,AK,6567,378435,NaN,36.0,NaN,NaN,NaN,8.0,NaN,...,385002,2,0,433e94e907f6c4815829cdad34dbf4c296918fc6,0,0,0,0,0,NaN
2020-09-05,AL,131803,846170,NaN,818.0,14914.0,NaN,1537.0,NaN,848.0,...,977973,9,113,b22e91650c4091000389f5e6f7d8cd4e4bb981eb,0,0,0,0,0,NaN
2020-09-05,AR,64690,700721,NaN,385.0,4456.0,NaN,NaN,78.0,566.0,...,765411,9,34,943b6e2742aae6e78aa190fb590e34d8804ccbd1,0,0,0,0,0,NaN
2020-09-05,AS,0,1514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1514,0,0,78615189f736f50add2170e1fb8468820dba264a,0,0,0,0,0,NaN
2020-09-05,AZ,205516,1047259,NaN,702.0,21520.0,225.0,NaN,131.0,NaN,...,1252775,36,22,f2b8f923b223050916eb45768429c30f9dce719c,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-09-05,AK,6567.0,378435.0,NaN,36.0,NaN,NaN,NaN,8.0,NaN,...,385002,2,0,433e94e907f6c4815829cdad34dbf4c296918fc6,0,0,0,0,0,NaN
2020-09-05,AL,131803.0,846170.0,NaN,818.0,14914.0,NaN,1537.0,NaN,848.0,...,977973,9,113,b22e91650c4091000389f5e6f7d8cd4e4bb981eb,0,0,0,0,0,NaN
2020-09-05,AR,64690.0,700721.0,NaN,385.0,4456.0,NaN,NaN,78.0,566.0,...,765411,9,34,943b6e2742aae6e78aa190fb590e34d8804ccbd1,0,0,0,0,0,NaN
2020-09-05,AS,0.0,1514.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1514,0,0,78615189f736f50add2170e1fb8468820dba264a,0,0,0,0,0,NaN
2020-09-05,AZ,205516.0,1047259.0,NaN,702.0,21520.0,225.0,NaN,131.0,NaN,...,1252775,36,22,f2b8f923b223050916eb45768429c30f9dce719c,0,0,0,0,0,NaN
